In [1]:
class Call:
    def __init__(self, op):
        self._op = op
        self._operand = []

    def __call__(self, *n):
        self._operand.extend(list(n))
        return self

def print_graph(call):
    done = {}
    def helper(call):
        if call not in done:
            args = []
            for n in call._operand:
                args.append(helper(n))
            done[call] = len(done) + 1
            print(f"%{done[call]} = {call._op}(" + ', '.join("%"+str(item) for item in args) + ")")
        return done[call]
    helper(call)

def node_commoning(call):
    cache = {}
    memo = {}
    def helper(node):
        if node is None:
            return None, None
        if node in memo:
            return memo[node]
        # Process operands
        operand_nodes = []
        operand_keys = []
        for opnd in node._operand:
            new_opnd, opnd_key = helper(opnd)
            operand_nodes.append(new_opnd)
            operand_keys.append(opnd_key)
        # Create key
        key = (node._op, tuple(operand_keys))
        # Check cache
        if key in cache:
            new_node = cache[key]
        else:
            new_node = Call(node._op)
            new_node._operand = operand_nodes
            cache[key] = new_node
        memo[node] = (new_node, key)
        return new_node, key
    new_call, _ = helper(call)
    return new_call

# Example Input Graph
input_node = Call('I')
c = Call('C')(Call('B')(Call('A')(input_node)))
d = Call('D')(Call('B')(Call('A')(input_node)))
output = Call('Z')(c, d)

print("Original Graph:")
print_graph(output)

mod_call = node_commoning(output)

print("\nModified Graph:")
print_graph(mod_call)


Original Graph:
%1 = I()
%2 = A(%1)
%3 = B(%2)
%4 = C(%3)
%5 = A(%1)
%6 = B(%5)
%7 = D(%6)
%8 = Z(%4, %7)

Modified Graph:
%1 = I()
%2 = A(%1)
%3 = B(%2)
%4 = C(%3)
%5 = D(%3)
%6 = Z(%4, %5)
